In [203]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score

import json


In [204]:

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NETURAL"
    POSITIVE = "POSITIVE"
class Review:
    def __init__ (self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <=2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #score of 4 5 
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__ (self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def  evenly_distrubute(self):
    
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews) ) 
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        

## Load Data 

In [229]:
## Load Data
import json 

file_name = 'C:/Users/fares/OneDrive/Desktop/Amazon Reviews/data/books_small_10000.json'
        
             
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

## PREP  DATA 

In [230]:


training,test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)

test_container= ReviewContainer(test)



In [231]:
train_container.evenly_distrubute()

train_x = train_container.get_text()

train_y = train_container.get_sentiment()


test_container.evenly_distrubute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

In [232]:
## print(training[0].sentiment)

## Bag of Vectorization

In [233]:
## fit and transform data 
## this book is great
## this book was so bad

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())


Wow! I don't even know how to put into words how amazing this book is. It's an experience you get to FEEL in your heart and soul! The words flow off each page and make you FEEL the humor, love and heartbreak that these characters are feeling. Add in the musical connection that's shared between Ridge and Sydney and you can FEEL the music they make together! Colleen Hoover has written an amazing work of art and I highly recommend everyone FEEL the experience of Maybe Someday!
[[0. 0. 0. ... 0. 0. 0.]]


Linear SVM

In [234]:
from sklearn import svm

clf_svm = svm.SVC(kernel="linear")

clf_svm.fit(train_x_vectors, train_y)

test_x[0]
 
clf_svm.predict(test_x_vectors[0])    
                  

array(['NEGATIVE'], dtype='<U8')

#### DECISION TREEE

In [235]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

    ## Naive Bayes

In [236]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

    #### Logestic regression 

In [237]:
from sklearn.linear_model import LogisticRegression
clf_log = DecisionTreeClassifier()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

## Evaluation

In [238]:
#Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_log.score(test_x_vectors,test_y))
print(clf_gnb.score(test_x_vectors,test_y))

0.8076923076923077
0.6201923076923077
0.6442307692307693
0.6394230769230769


In [239]:
#F1 Scores 
from sklearn.metrics import f1_score

f1_score(test_y,clf_svm.predict(test_x_vectors), average = None, labels =[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
#f1_score(test_y,clf_gnb.predict(test_x_vectors), average = None, labels =[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
#f1_score(test_y,clf_log.predict(test_x_vectors), average = None, labels =[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
#f1_score(test_y,clf_dec.predict(test_x_vectors), average = None, labels =[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])


C:\Users\fares\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


array([0.80582524, 0.        , 0.80952381])

In [240]:
test_set = [' amazing', 'bad book do not buy', 'loved it']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'POSITIVE'], dtype='<U8')

## Tuning the Model using GridSearchCV

In [241]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)
 


GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [242]:
print(clf.score(test_x_vectors,test_y))

0.8052884615384616
